In [1]:
def create_rf3_inputs(seq, name):
    return {
        "name": name,
        "components": [
            {
                "seq": seq,
                "chain_id": ["A", "B"],
            }
        ]
    }

### ANALYZE

In [2]:
import json
import prody as pr
import numpy as np
import pandas as pd
from pathlib import Path

paths = [ './pose_00', './pose_01', './pose_02', './pose_03', './pose_04', './pose_05']

all_rf3_inputs = []
all_data = []
for path in paths:
    path = Path(path)
    for pdb in sorted(list(path.glob('*.pdb'))):
        if 'monomer' not in pdb.stem:
            continue
        seq = pr.parsePDB(str(pdb)).ca.getSequence()
        all_data.append({'rosetta_emin_monomer_path': str(pdb), 'sequence': seq})

        name = f'{path}-{pdb.stem}'
        all_rf3_inputs.append(create_rf3_inputs(seq, name))

chunks = np.array_split(all_rf3_inputs, 8)
for cidx, chunk in enumerate(chunks):
    opath = Path(f'./rf3_inputs_{cidx}.json')
    with opath.open('w') as f:
        json.dump(list(chunk), f, indent=2)

df = pd.DataFrame(all_data)
df

rosetta_emin_monomer_path  \
0     pose_00/000_monomer.pdb   
1     pose_00/001_monomer.pdb   
2     pose_00/002_monomer.pdb   
3     pose_00/003_monomer.pdb   
4     pose_00/004_monomer.pdb   
..                        ...   
412   pose_05/053_monomer.pdb   
413   pose_05/054_monomer.pdb   
414   pose_05/055_monomer.pdb   
415   pose_05/056_monomer.pdb   
416   pose_05/057_monomer.pdb   

                                              sequence  
0    APSMVEQAKQLGSQLVEAANSGDTEKVRSLLAPDATLTYRDGTKLT...  
1    KPSLKDQAKQLGKDFVEAANKGDSDKIESMLAPNATMKLHDGTELK...  
2    APSISDQARQLGQEFVEAVNKGDEEKVRAMLAPDATMKLSDGTVLT...  
3    EPSLVERARRLGREFVEAANSGDTEKVRSLLAPDATLTLSDGTRLR...  
4    APSLKEQAKELGKLFVEAANEGDDERVKSMLAPDATMKLSDGTVLT...  
..                                                 ...  
412  GHLPSNAEKARSLTQQVFEAYNSGDFEKLSSLFSDDFYAVDGEGKR...  
413  MDKEDLAKEAVTLTREVFEAYNSGDFDKLNELFADDFFAQNGQGQR...  
414  PLEEESGEDAIKLTQQVFEAYNSGDFDKLRELFADDFYATDGSGQR...  
415  GPSTSLGSQAVSLTQQIFDAYNSGDFDLLSQLFADDFYAENGQGQR...  
416  HAHEERGEEAIKLTKEVFEAYNSGDFEKLKELFADDFYAVDGSGKR...  

[417 rows x 2 columns]

In [3]:
!cd rf3_outputs; gunzip *.gz

gzip: *.gz: No such file or directory


In [4]:
all_data = []
for path in Path('./rf3_outputs/').glob('*.cif'):
    protein = pr.parseMMCIF(str(path))
    seq = protein.select('chid A and name CA').getSequence()
    all_data.append({'sequence': seq, 'rf3_dimer_path': str(path)})

df2 = pd.DataFrame(all_data)
df2

sequence  \
0    MSDEEKEEEAKSLVKEFFDAVNSGDVATVKEILAESYVMTDGTGTK...   
1    ASTEEQQEIAKQYAEALNQGDIDSLLSLFAPDAVITTDDGTQLTSL...   
2    PDKEKAAAEAVKLTQQVFEAYNSGDFDKLKELFADDFYATDGEGKR...   
3    MSDEEKRSIAEQYAQALNEGDIDKLLSLYAPDAVVTTDDGTQLTSL...   
4    MSDEEKKEEARTLVHDFFDAVNSGDKEKIEEILADDYVMTDGKGTE...   
..                                                 ...   
412  PSNEEMEQIANEYAEALNEGDIEKLLALFAPDATVTDEDGTVYTSL...   
413  AALPSMSELAKALTQQVFQAYNSGDFDLLKSLFSDDFFAIDGKGRK...   
414  EPSLRDQARQLGKEFVEAANAGDREKVRSMLAPDATMKLADGTVLE...   
415  GSQPSNAQQAQALTQQVFDAINSGDFEKLRSLFADDFFAIDGSGNR...   
416  PSEQEAREIAEDYAEALNRGDIDKLLSLFAPDATVVTSDGKKLTSL...   

                                  rf3_dimer_path  
0    rf3_outputs/pose_02-040_monomer_model_0.cif  
1    rf3_outputs/pose_01-054_monomer_model_0.cif  
2    rf3_outputs/pose_05-040_monomer_model_0.cif  
3    rf3_outputs/pose_01-055_monomer_model_0.cif  
4    rf3_outputs/pose_02-002_monomer_model_0.cif  
..                                           ...  
412  rf3_outputs/pose_01-034_monomer_model_0.cif  
413  rf3_outputs/pose_05-042_monomer_model_0.cif  
414  rf3_outputs/pose_00-040_monomer_model_0.cif  
415  rf3_outputs/pose_05-004_monomer_model_0.cif  
416  rf3_outputs/pose_01-011_monomer_model_0.cif  

[417 rows x 2 columns]

In [5]:
df = pd.merge(df, df2, on='sequence')
df

rosetta_emin_monomer_path  \
0     pose_00/000_monomer.pdb   
1     pose_00/001_monomer.pdb   
2     pose_00/002_monomer.pdb   
3     pose_00/003_monomer.pdb   
4     pose_00/004_monomer.pdb   
..                        ...   
412   pose_05/053_monomer.pdb   
413   pose_05/054_monomer.pdb   
414   pose_05/055_monomer.pdb   
415   pose_05/056_monomer.pdb   
416   pose_05/057_monomer.pdb   

                                              sequence  \
0    APSMVEQAKQLGSQLVEAANSGDTEKVRSLLAPDATLTYRDGTKLT...   
1    KPSLKDQAKQLGKDFVEAANKGDSDKIESMLAPNATMKLHDGTELK...   
2    APSISDQARQLGQEFVEAVNKGDEEKVRAMLAPDATMKLSDGTVLT...   
3    EPSLVERARRLGREFVEAANSGDTEKVRSLLAPDATLTLSDGTRLR...   
4    APSLKEQAKELGKLFVEAANEGDDERVKSMLAPDATMKLSDGTVLT...   
..                                                 ...   
412  GHLPSNAEKARSLTQQVFEAYNSGDFEKLSSLFSDDFYAVDGEGKR...   
413  MDKEDLAKEAVTLTREVFEAYNSGDFDKLNELFADDFFAQNGQGQR...   
414  PLEEESGEDAIKLTQQVFEAYNSGDFDKLRELFADDFYATDGSGQR...   
415  GPSTSLGSQAVSLTQQIFDAYNSGDFDLLSQLFADDFYAENGQGQR...   
416  HAHEERGEEAIKLTKEVFEAYNSGDFEKLKELFADDFYAVDGSGKR...   

                                  rf3_dimer_path  
0    rf3_outputs/pose_00-000_monomer_model_0.cif  
1    rf3_outputs/pose_00-001_monomer_model_0.cif  
2    rf3_outputs/pose_00-002_monomer_model_0.cif  
3    rf3_outputs/pose_00-003_monomer_model_0.cif  
4    rf3_outputs/pose_00-004_monomer_model_0.cif  
..                                           ...  
412  rf3_outputs/pose_05-053_monomer_model_0.cif  
413  rf3_outputs/pose_05-054_monomer_model_0.cif  
414  rf3_outputs/pose_05-055_monomer_model_0.cif  
415  rf3_outputs/pose_05-056_monomer_model_0.cif  
416  rf3_outputs/pose_05-057_monomer_model_0.cif  

[417 rows x 3 columns]

In [6]:
import numpy as np

rmsds = []
for i, row in df.iterrows():
    monomer = pr.parsePDB(str(row['rosetta_emin_monomer_path']))
    dimer = pr.parseMMCIF(str(row['rf3_dimer_path']))

    metadata_path = Path(row['rf3_dimer_path']).parent / f"{Path(row['rf3_dimer_path']).stem.replace('_model_0', '_metrics')}.csv"
    iptm = pd.read_csv(metadata_path)['iptm.iptm_0'].iloc[0]

    metadata_path2 = Path(row['rf3_dimer_path']).parent / f"{Path(row['rf3_dimer_path']).stem.replace('_model_0', '')}.score"
    interface_pae = pd.read_csv(metadata_path2).pae_interface.dropna().iloc[0]

    monomer = pr.calcTransformation(monomer.select('chid A and name CA'), dimer.select('chid A and name CA')).apply(monomer)
    rmsd = pr.calcRMSD(monomer.select('chid A and name CA'), dimer.select('chid A and name CA'))
    sequence = monomer.ca.getSequence()
    rmsds.append({'ca_rmsds': rmsd, 'sequence': sequence, 'interface_pae': interface_pae, 'iptm': iptm})

df = df.merge(pd.DataFrame(rmsds), on='sequence')

In [7]:
filtered_df = df[(df.interface_pae > 15) & (df.ca_rmsds < 1.0)].copy()
filtered_df

rosetta_emin_monomer_path  \
3     pose_00/003_monomer.pdb   
5     pose_00/005_monomer.pdb   
12    pose_00/012_monomer.pdb   
13    pose_00/013_monomer.pdb   
18    pose_00/018_monomer.pdb   
..                        ...   
379   pose_05/020_monomer.pdb   
391   pose_05/032_monomer.pdb   
392   pose_05/033_monomer.pdb   
393   pose_05/034_monomer.pdb   
395   pose_05/036_monomer.pdb   

                                              sequence  \
3    EPSLVERARRLGREFVEAANSGDTEKVRSLLAPDATLTLSDGTRLR...   
5    PPSLEEQARSLGEEFVSAANSGDKSSVSSMLAPDATMTLHDGTKLT...   
12   APSLQSQAETLGKEFIDAANEGDVEKMRSMLAPDATMKLDDGTELK...   
13   LPSLVERARELGKELVEAANSGDTEKVKSLLSPDATLTLADGTKLT...   
18   APSLEDQARQLGREFVEAANQGDEEKIRSMLAPDATMKLADGTELK...   
..                                                 ...   
379  GSMPSMGEQARSLTQQVFEAYNSGNFDKLASLFADDFFAVDENGRV...   
391  PDHQESADLAIQLTQEVFDAYNSGDFDKLSELFADDFYAVDGSGQR...   
392  PALQQAADQAVQLTQEIFEAYNSGDFEKLKELFADDFYATDGTGER...   
393  PAEEEAAQEAVELTRQVFEAYNSGNFEKLSELFADDFFAEDGSGKR...   
395  MSHMEAAEEAKKLTKEVFEAYNSGDFDKLKELFASDFYAEDGSGTR...   

                                  rf3_dimer_path  ca_rmsds  interface_pae  \
3    rf3_outputs/pose_00-003_monomer_model_0.cif  0.399440      15.901764   
5    rf3_outputs/pose_00-005_monomer_model_0.cif  0.564744      16.567131   
12   rf3_outputs/pose_00-012_monomer_model_0.cif  0.509034      16.988491   
13   rf3_outputs/pose_00-013_monomer_model_0.cif  0.577893      15.609126   
18   rf3_outputs/pose_00-018_monomer_model_0.cif  0.475083      16.050146   
..                                           ...       ...            ...   
379  rf3_outputs/pose_05-020_monomer_model_0.cif  0.988571      16.463816   
391  rf3_outputs/pose_05-032_monomer_model_0.cif  0.812859      15.431754   
392  rf3_outputs/pose_05-033_monomer_model_0.cif  0.488264      15.289605   
393  rf3_outputs/pose_05-034_monomer_model_0.cif  0.447252      17.431313   
395  rf3_outputs/pose_05-036_monomer_model_0.cif  0.418320      15.768366   

         iptm  
3    0.396546  
5    0.372394  
12   0.369329  
13   0.416855  
18   0.399199  
..        ...  
379  0.383963  
391  0.417721  
392  0.418306  
393  0.392181  
395  0.406414  

[135 rows x 6 columns]

In [8]:
data = []

for idx, row in filtered_df.iterrows():
    monomer_path = Path(row.rosetta_emin_monomer_path).absolute()
    rf3_dimer_path = Path(row.rf3_dimer_path).absolute()

    print(str(monomer_path.absolute()))

    monomer = pr.parsePDB(str(monomer_path))
    dimer = pr.parsePDB(str(rf3_dimer_path))
    monomer = pr.calcTransformation(monomer.select('name CA and chid A'), dimer.select('name CA and chid A')).apply(monomer)
    binding_site_resnums = monomer.select('(same residue as ((protein and not element H and not name N CA C O) within 5.0 of (chid B and not element H))) and name CA').getResnums()
    assert (monomer.select('chid A and (not element H) and resnum '+' '.join([str(int(x)) for x in binding_site_resnums])).getNames() == dimer.select('chid A and resnum '+' '.join([str(int(x)) for x in binding_site_resnums])).getNames()).all(), f'Atom Name order is mismatched for scRMSD calculation.'

    scRMSD = pr.calcRMSD(monomer.select('chid A and (not element H) and resnum '+' '.join([str(int(x)) for x in binding_site_resnums])), dimer.select('chid A and resnum '+' '.join([str(int(x)) for x in binding_site_resnums])))
    sequence = monomer.ca.getSequence()

    data.append({'sidechain_rmsd': scRMSD, 'sequence': sequence})
df2 = pd.DataFrame(data)
filtered_df = filtered_df.merge(df2, on='sequence')

/scratch/bfry/exxact_server_data/bfry/apixaban_nise/af2_selected/pose_00/003_monomer.pdb
/scratch/bfry/exxact_server_data/bfry/apixaban_nise/af2_selected/pose_00/005_monomer.pdb
/scratch/bfry/exxact_server_data/bfry/apixaban_nise/af2_selected/pose_00/012_monomer.pdb
/scratch/bfry/exxact_server_data/bfry/apixaban_nise/af2_selected/pose_00/013_monomer.pdb
/scratch/bfry/exxact_server_data/bfry/apixaban_nise/af2_selected/pose_00/018_monomer.pdb
/scratch/bfry/exxact_server_data/bfry/apixaban_nise/af2_selected/pose_00/019_monomer.pdb
/scratch/bfry/exxact_server_data/bfry/apixaban_nise/af2_selected/pose_00/021_monomer.pdb
/scratch/bfry/exxact_server_data/bfry/apixaban_nise/af2_selected/pose_00/022_monomer.pdb
/scratch/bfry/exxact_server_data/bfry/apixaban_nise/af2_selected/pose_00/025_monomer.pdb
/scratch/bfry/exxact_server_data/bfry/apixaban_nise/af2_selected/pose_00/039_monomer.pdb
/scratch/bfry/exxact_server_data/bfry/apixaban_nise/af2_selected/pose_00/044_monomer.pdb
/scratch/bfry/exxact_

In [12]:
filtered_df[(filtered_df.interface_pae > 15.0) & (filtered_df.ca_rmsds < 1.0) & (filtered_df.rosetta_emin_monomer_path.map(lambda x: x.split('/')[0]) == 'pose_00')].iloc[:6].sequence.tolist()

['EPSLVERARRLGREFVEAANSGDTEKVRSLLAPDATLTLSDGTRLRKREDIVEHYRTGYRAGLTVRVDVQSVNLLDPDRALARARGVVSSAATGEVRHLDTEALFELIDGELFIKRLSGRLAP',
 'PPSLEEQARSLGEEFVSAANSGDKSSVSSMLAPDATMTLHDGTKLTRKSDITEYYKQGYENGYKVSIDIKSVEQIGADEAVAEAEGTVSSSKTGEVVKLKTRAKFVLVDGKVYIKDLEGEELP',
 'APSLQSQAETLGKEFIDAANEGDVEKMRSMLAPDATMKLDDGTELKKREDIVEYYKDGYENGYKVEVTIESVEVVGSDRAVATAEGTVRSTKTGEVVKLRTKAEFRLVNGQVYILRLEGERLP',
 'LPSLVERARELGKELVEAANSGDTEKVKSLLSPDATLTLADGTKLTKRDDIVDYYRKGHEAGLSVEVKIRSVELLGPNRALARADGVVSSSATGDVTRLKTSALFELIDGKVFVKELSGELAP',
 'APSLEDQARQLGREFVEAANQGDEEKIRSMLAPDATMKLADGTELKKKDDIVKFYKEGYENGYKVDVEILSVEVLGSDRARARARGTVRSEKTGEVVRLETRADFRLVNGKVYILRLEGRRLP',
 'MPSLEEQARELGEELVDAMNEGDVEKVRSMLAPDATMTLSNGEELTAREDIVEYYQEGFENGYRIEVDVLSVEVIGADRARARANGTVTSEKTGEVIKLRTEAEFRLVDGEVYVHRLRGEQIA']

In [9]:
A = filtered_df[(filtered_df.interface_pae > 15.0) & (filtered_df.ca_rmsds < 1.0) & (filtered_df.sidechain_rmsd < 0.7)].copy()
A['pose'] = A.rosetta_emin_monomer_path.map(lambda x: Path(x).parent)
A

rosetta_emin_monomer_path  \
2     pose_00/012_monomer.pdb   
7     pose_00/022_monomer.pdb   
10    pose_00/044_monomer.pdb   
16    pose_01/000_monomer.pdb   
18    pose_01/004_monomer.pdb   
22    pose_01/013_monomer.pdb   
24    pose_01/017_monomer.pdb   
26    pose_01/021_monomer.pdb   
28    pose_01/023_monomer.pdb   
34    pose_01/039_monomer.pdb   
35    pose_01/044_monomer.pdb   
38    pose_01/051_monomer.pdb   
41    pose_01/060_monomer.pdb   
42    pose_01/061_monomer.pdb   
45    pose_01/068_monomer.pdb   
46    pose_01/069_monomer.pdb   
48    pose_01/075_monomer.pdb   
49    pose_01/076_monomer.pdb   
51    pose_01/079_monomer.pdb   
52    pose_01/085_monomer.pdb   
58    pose_02/020_monomer.pdb   
61    pose_02/044_monomer.pdb   
85    pose_03/013_monomer.pdb   
87    pose_03/023_monomer.pdb   
88    pose_03/024_monomer.pdb   
89    pose_03/025_monomer.pdb   
92    pose_03/033_monomer.pdb   
93    pose_03/034_monomer.pdb   
95    pose_03/038_monomer.pdb   
96    pose_03/039_monomer.pdb   
97    pose_03/044_monomer.pdb   
99    pose_03/048_monomer.pdb   
100   pose_03/052_monomer.pdb   

                                              sequence  \
2    APSLQSQAETLGKEFIDAANEGDVEKMRSMLAPDATMKLDDGTELK...   
7    LPSLEDQARTLGKKFVQAANSGNTDQVRSMLAPDATMTLADGTQLN...   
10   GPSIEEQAKALGQQFVDAANSGNVEQVKSLLSPDATLTLADGTKLS...   
16   MSESEQRQIALDYAQALNDGDVESLLSLFAPDAVITLADGTKLTSL...   
18   PSNEEMREIAKEYAQALNDGDIDALLSLFAPDATIVTSDGTVLTSL...   
22   MSDDDMERVAQAYADALNEGDIDRLLSLFAPDAIITTSDGTKLESL...   
24   MSQSQQRSIAQQYAQALNDGDIDSLLSLFAPDAVIVQSDGTKLTSL...   
26   PSEEEARKIAEEYAAALNEGDIDRLLSLFAPDATITTEDGRKLTSL...   
28   MSQEEQRSIALQYAQALNDGDIDSLLSLFAPDATITTDDGTKLNSL...   
34   MSQQQQKQIAQDYAQALNDGDIDALLSLFAPDAVITTADGTKLTSL...   
35   PSDQEMRDIATRYAEALNRGDIEELLSLFAPDATIRTSDGTLLTSL...   
38   MSVEEQKSIALQYAQALNDGDIDSLLDLFAPDAVVTTSDGTELKSL...   
41   PSNEEMKDIAESYAEALNEGDIDKLLSLFAPNATITTSDGTRLTSL...   
42   PSDEEKKSIAQSFAKAMNEGDIDSLLSLFAPKATITTADGQLLTSL...   
45   PSDEQMRQIAQEYAEALNKGDIDRLLSLFAPDATITTDDGTKLTSL...   
46   PSKEEMRKIAESYAQALNRGDIDKLLSLFAPDATITTSDGTLLTSL...   
48   PSDQEAKDIAQKYAEALNKGDIDTLLSLFAPDATITTSDGTLLTSL...   
49   PSDEEMKQIAQSYAEALNKGNIDRLLSLFAPDATITTSDGTLLTSL...   
51   MSEDEAKEIAKQYAEALNQGDIDKLLSLFAPDAVITTDDGTKLTSL...   
52   PSQEEMRRIAEEYAEALNRGDIDKLLSLFAPDATITTSDGTLLTSL...   
58   MTKEELEKRAVEMVHQFFDAVNEGDIEEVRRILAEDYEMVDGTGKK...   
61   MSKEEKAKEAKKLVVEFFDAVNSGDVATVKRILDSDYKMKDGTGTE...   
85   MSKEEVLAKLDRWVDAVNSGDLDLLRSMAEPDATVTLPDGSVLGSR...   
87   MSKDEALSLLEQFIDAVNSGNLEKLKSMADPDATVRLPDGSVLNSV...   
88   MSKEEALELLKKFVDAVNSGDLEKLRSMADPNATVKLPDGSVLNSV...   
89   MSEEEALKLLRTFVDAVNSGNLELLRSMADPNATVTLPDGSVLSTV...   
92   MSDEEALELLKQFVDAVNSGDLDKLRSMADPNATVTLPDGSVLNSV...   
93   MSDEEALELLQEFVDAVNSGDLEKLKSMADPNATVVLPDGSVLTTV...   
95   MSREEALALLNRFVKAVNSGDLEELRSMADPNATVTLPDGSVLQSV...   
96   MSDDEALALLDEFVDAVNSGNLEKLKSMADPNATVRLPDGSILKSK...   
97   MSREEALSLLQQFVDAVNSGNLEQLKSMADPNATVTLPDGSVLESV...   
99   MSDEEALSLLKKFVDAVNSGDLDELKSMADPDATVTLPDGSVLESV...   
100  MSKEEALEKLKEFVDAVNKGDLASLEEMAEPDATVTLPDGSVLTSK...   

                                  rf3_dimer_path  ca_rmsds  interface_pae  \
2    rf3_outputs/pose_00-012_monomer_model_0.cif  0.509034      16.988491   
7    rf3_outputs/pose_00-022_monomer_model_0.cif  0.470406      15.951316   
10   rf3_outputs/pose_00-044_monomer_model_0.cif  0.605280      17.072321   
16   rf3_outputs/pose_01-000_monomer_model_0.cif  0.452824      16.975536   
18   rf3_outputs/pose_01-004_monomer_model_0.cif  0.257887      18.676342   
22   rf3_outputs/pose_01-013_monomer_model_0.cif  0.417518      15.980603   
24   rf3_outputs/pose_01-017_monomer_model_0.cif  0.342152      15.305738   
26   rf3_outputs/pose_01-021_monomer_model_0.cif  0.585412      16.747427   
28   rf3_outputs/pose_01-023_monomer_model_0.cif  0.369091      15.477242   
34   rf3_outputs/pose_01-039_monomer_model_0.cif  0.372391      17.852890   
35   rf3_outputs/pose_01-044_monomer_model_0.cif  0.308449   

In [14]:
filtered_df[(filtered_df.interface_pae > 15.0) & (filtered_df.ca_rmsds < 1.0) & (filtered_df.rosetta_emin_monomer_path.map(lambda x: x.split('/')[0].stem == 'pose_00'))].sort_values('sidechain_rmsd', ascending=True)

AttributeError: 'str' object has no attribute 'stem'

In [10]:
filtered_df[(filtered_df.interface_pae > 15.0) & (filtered_df.ca_rmsds < 1.0) & (filtered_df.sidechain_rmsd < 0.7)]

rosetta_emin_monomer_path  \
2     pose_00/012_monomer.pdb   
7     pose_00/022_monomer.pdb   
10    pose_00/044_monomer.pdb   
16    pose_01/000_monomer.pdb   
18    pose_01/004_monomer.pdb   
22    pose_01/013_monomer.pdb   
24    pose_01/017_monomer.pdb   
26    pose_01/021_monomer.pdb   
28    pose_01/023_monomer.pdb   
34    pose_01/039_monomer.pdb   
35    pose_01/044_monomer.pdb   
38    pose_01/051_monomer.pdb   
41    pose_01/060_monomer.pdb   
42    pose_01/061_monomer.pdb   
45    pose_01/068_monomer.pdb   
46    pose_01/069_monomer.pdb   
48    pose_01/075_monomer.pdb   
49    pose_01/076_monomer.pdb   
51    pose_01/079_monomer.pdb   
52    pose_01/085_monomer.pdb   
58    pose_02/020_monomer.pdb   
61    pose_02/044_monomer.pdb   
85    pose_03/013_monomer.pdb   
87    pose_03/023_monomer.pdb   
88    pose_03/024_monomer.pdb   
89    pose_03/025_monomer.pdb   
92    pose_03/033_monomer.pdb   
93    pose_03/034_monomer.pdb   
95    pose_03/038_monomer.pdb   
96    pose_03/039_monomer.pdb   
97    pose_03/044_monomer.pdb   
99    pose_03/048_monomer.pdb   
100   pose_03/052_monomer.pdb   

                                              sequence  \
2    APSLQSQAETLGKEFIDAANEGDVEKMRSMLAPDATMKLDDGTELK...   
7    LPSLEDQARTLGKKFVQAANSGNTDQVRSMLAPDATMTLADGTQLN...   
10   GPSIEEQAKALGQQFVDAANSGNVEQVKSLLSPDATLTLADGTKLS...   
16   MSESEQRQIALDYAQALNDGDVESLLSLFAPDAVITLADGTKLTSL...   
18   PSNEEMREIAKEYAQALNDGDIDALLSLFAPDATIVTSDGTVLTSL...   
22   MSDDDMERVAQAYADALNEGDIDRLLSLFAPDAIITTSDGTKLESL...   
24   MSQSQQRSIAQQYAQALNDGDIDSLLSLFAPDAVIVQSDGTKLTSL...   
26   PSEEEARKIAEEYAAALNEGDIDRLLSLFAPDATITTEDGRKLTSL...   
28   MSQEEQRSIALQYAQALNDGDIDSLLSLFAPDATITTDDGTKLNSL...   
34   MSQQQQKQIAQDYAQALNDGDIDALLSLFAPDAVITTADGTKLTSL...   
35   PSDQEMRDIATRYAEALNRGDIEELLSLFAPDATIRTSDGTLLTSL...   
38   MSVEEQKSIALQYAQALNDGDIDSLLDLFAPDAVVTTSDGTELKSL...   
41   PSNEEMKDIAESYAEALNEGDIDKLLSLFAPNATITTSDGTRLTSL...   
42   PSDEEKKSIAQSFAKAMNEGDIDSLLSLFAPKATITTADGQLLTSL...   
45   PSDEQMRQIAQEYAEALNKGDIDRLLSLFAPDATITTDDGTKLTSL...   
46   PSKEEMRKIAESYAQALNRGDIDKLLSLFAPDATITTSDGTLLTSL...   
48   PSDQEAKDIAQKYAEALNKGDIDTLLSLFAPDATITTSDGTLLTSL...   
49   PSDEEMKQIAQSYAEALNKGNIDRLLSLFAPDATITTSDGTLLTSL...   
51   MSEDEAKEIAKQYAEALNQGDIDKLLSLFAPDAVITTDDGTKLTSL...   
52   PSQEEMRRIAEEYAEALNRGDIDKLLSLFAPDATITTSDGTLLTSL...   
58   MTKEELEKRAVEMVHQFFDAVNEGDIEEVRRILAEDYEMVDGTGKK...   
61   MSKEEKAKEAKKLVVEFFDAVNSGDVATVKRILDSDYKMKDGTGTE...   
85   MSKEEVLAKLDRWVDAVNSGDLDLLRSMAEPDATVTLPDGSVLGSR...   
87   MSKDEALSLLEQFIDAVNSGNLEKLKSMADPDATVRLPDGSVLNSV...   
88   MSKEEALELLKKFVDAVNSGDLEKLRSMADPNATVKLPDGSVLNSV...   
89   MSEEEALKLLRTFVDAVNSGNLELLRSMADPNATVTLPDGSVLSTV...   
92   MSDEEALELLKQFVDAVNSGDLDKLRSMADPNATVTLPDGSVLNSV...   
93   MSDEEALELLQEFVDAVNSGDLEKLKSMADPNATVVLPDGSVLTTV...   
95   MSREEALALLNRFVKAVNSGDLEELRSMADPNATVTLPDGSVLQSV...   
96   MSDDEALALLDEFVDAVNSGNLEKLKSMADPNATVRLPDGSILKSK...   
97   MSREEALSLLQQFVDAVNSGNLEQLKSMADPNATVTLPDGSVLESV...   
99   MSDEEALSLLKKFVDAVNSGDLDELKSMADPDATVTLPDGSVLESV...   
100  MSKEEALEKLKEFVDAVNKGDLASLEEMAEPDATVTLPDGSVLTSK...   

                                  rf3_dimer_path  ca_rmsds  interface_pae  \
2    rf3_outputs/pose_00-012_monomer_model_0.cif  0.509034      16.988491   
7    rf3_outputs/pose_00-022_monomer_model_0.cif  0.470406      15.951316   
10   rf3_outputs/pose_00-044_monomer_model_0.cif  0.605280      17.072321   
16   rf3_outputs/pose_01-000_monomer_model_0.cif  0.452824      16.975536   
18   rf3_outputs/pose_01-004_monomer_model_0.cif  0.257887      18.676342   
22   rf3_outputs/pose_01-013_monomer_model_0.cif  0.417518      15.980603   
24   rf3_outputs/pose_01-017_monomer_model_0.cif  0.342152      15.305738   
26   rf3_outputs/pose_01-021_monomer_model_0.cif  0.585412      16.747427   
28   rf3_outputs/pose_01-023_monomer_model_0.cif  0.369091      15.477242   
34   rf3_outputs/pose_01-039_monomer_model_0.cif  0.372391      17.852890   
35   rf3_outputs/pose_01-044_monomer_model_0.cif  0.308449   

In [ ]:
import shutil

path = Path('./apx_top_by_af2_and_rf3/')
path.mkdir(exist_ok=True)

for idx, row in filtered_df[(filtered_df.interface_pae > 15.0) & (filtered_df.ca_rmsds < 1.0) & (filtered_df.sidechain_rmsd < 0.7)].copy().iterrows():
    # print(row.boltz_dimer_path, Path(row.rosetta_emin_monomer_path).stem)

    shutil.copy(row.rf3_dimer_path, path / f'rf3_dimer_{Path(row.rosetta_emin_monomer_path).stem.replace("_monomer", "")}.cif')
    shutil.copy(row.rosetta_emin_monomer_path, path / f'boltz_monomer_{Path(row.rosetta_emin_monomer_path).stem.rsplit("_monomer", 1)[0]}.pdb')
    shutil.copy(Path(row.rosetta_emin_monomer_path).parent / f'{Path(row.rosetta_emin_monomer_path).stem.replace("monomer", "af2_dimer")}.pdb', path / f'af2_dimer_{Path(row.rosetta_emin_monomer_path).stem.split("_")[0]}.pdb')

In [26]:
import prody as pr
for boltz_monomer in sorted(list(path.glob('boltz_monomer*.pdb'))):
    seq = pr.parsePDB(str(boltz_monomer)).ca.getSequence()

    opath = path / 'boltz_inputs' / f"{boltz_monomer.stem}.fasta"
    with opath.open('w') as f:
        f.write(f">A|protein|empty\n{seq}\n>B|protein|empty\n{seq}\n")

@> 1872 atoms and 1 coordinate set(s) were parsed in 0.02s.
@> 1913 atoms and 1 coordinate set(s) were parsed in 0.02s.
@> 1977 atoms and 1 coordinate set(s) were parsed in 0.02s.
@> 1917 atoms and 1 coordinate set(s) were parsed in 0.02s.
@> 1863 atoms and 1 coordinate set(s) were parsed in 0.01s.


@> 2025 atoms and 1 coordinate set(s) were parsed in 0.02s.
@> 1907 atoms and 1 coordinate set(s) were parsed in 0.02s.
@> 1915 atoms and 1 coordinate set(s) were parsed in 0.02s.
@> 1855 atoms and 1 coordinate set(s) were parsed in 0.01s.
@> 1871 atoms and 1 coordinate set(s) were parsed in 0.01s.
@> 1884 atoms and 1 coordinate set(s) were parsed in 0.01s.
@> 1862 atoms and 1 coordinate set(s) were parsed in 0.01s.
@> 1892 atoms and 1 coordinate set(s) were parsed in 0.01s.
@> 1894 atoms and 1 coordinate set(s) were parsed in 0.01s.
@> 1908 atoms and 1 coordinate set(s) were parsed in 0.01s.
@> 1877 atoms and 1 coordinate set(s) were parsed in 0.01s.
@> 1861 atoms and 1 coordinate set(s) were parsed in 0.01s.
@> 1867 atoms and 1 coordinate set(s) were parsed in 0.01s.
@> 1932 atoms and 1 coordinate set(s) were parsed in 0.02s.
@> 1879 atoms and 1 coordinate set(s) were parsed in 0.01s.
@> 1897 atoms and 1 coordinate set(s) were parsed in 0.01s.
@> 1967 atoms and 1 coordinate set(s) we